# CSV Preprocess Job
入力CSVを読み込み、前処理して出力CSVを書き出します。`papermill` から以下のパラメータが渡されます。
- input_csv: 入力ファイルパス
- output_csv: 出力ファイルパス
- skip_rows: 先頭からスキップする行数（任意）
- header_row: ヘッダ行の0始まりインデックス（任意、指定時はnames推論に使う）
- rename_map: 列名の置換辞書（任意）
- filters: 行フィルタ条件の配列（任意、{col, op, value}）
- dtypes: 列ごとの型指定（任意）

In [ ]:
# Parameters
input_csv = None
output_csv = None
skip_rows = 0
header_row = None
rename_map = {}
filters = []
dtypes = {}

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path

assert input_csv is not None and output_csv is not None, "input_csv と output_csv は必須です"

read_kwargs = dict(encoding='utf-8', engine='python', on_bad_lines='skip')
if header_row is not None:
    read_kwargs.update(dict(skiprows=header_row + 1))
if skip_rows and (header_row is None):
    read_kwargs.update(dict(skiprows=skip_rows))

df = pd.read_csv(input_csv, **read_kwargs)

# 列名のクリーニングとリネーム
cols = []
seen = {}
for c in [str(x) for x in df.columns]:
    base = c.strip().replace(' ', '_') or 'col'
    if base in seen:
        seen[base] += 1
        base = f"{base}_{seen[base]}"
    else:
        seen[base] = 1
    cols.append(base)

df.columns = cols

if rename_map:
    df = df.rename(columns=rename_map)

# 型変換
for col, typ in (dtypes or {}).items():
    if col not in df.columns:
        continue
    try:
        if typ.lower() in ('int', 'integer'):
            df[col] = pd.to_numeric(df[col], errors='coerce').astype('Int64')
        elif typ.lower() in ('float', 'double'):
            df[col] = pd.to_numeric(df[col], errors='coerce')
        elif typ.lower() in ('datetime', 'timestamp'):
            df[col] = pd.to_datetime(df[col], errors='coerce')
        elif typ.lower() in ('bool', 'boolean'):
            df[col] = df[col].astype('boolean')
        else:
            df[col] = df[col].astype(str)
    except Exception:
        pass

# 行フィルタ
ops = {
    'eq': lambda a, b: a == b,
    'contains': lambda a, b: (str(b).lower() in str(a).lower()) if a is not None else False,
    'gte': lambda a, b: (a is not None) and (float(a) >= float(b)),
    'lte': lambda a, b: (a is not None) and (float(a) <= float(b)),
}

for cond in (filters or []):
    col = cond.get('col')
    op = cond.get('op', 'eq')
    val = cond.get('value')
    fn = ops.get(op)
    if not fn or col not in df.columns:
        continue
    mask = df[col].map(lambda x: fn(x, val))
    df = df[mask]

# 出力
Path(output_csv).parent.mkdir(parents=True, exist_ok=True)
df.to_csv(output_csv, index=False)
print(f"Wrote: {output_csv}, rows={len(df)}, cols={len(df.columns)}")